In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import torch.nn.functional as F


In [ ]:
os.chdir("../")
from src.data.request_data import getDataTF, getDataHF
from src.data.processing_data import Format
from src.models.pipelines import Pipeline
from src.models.encoders import BERTencoder

In [ ]:
df = pd.read_csv(f"./inputs_data/data_mrda_train.csv", encoding="utf-8",  sep="|")

In [ ]:
df.head(10)

In [ ]:
# number of distinct label i the dataset
len(df["Label"].unique())

In [ ]:
# number of distinct dialogue i the dataset
len(df["Dialogue_ID"].unique())

In [ ]:
# number of utterance per dialogue
# df[["Dialogue_ID", "Label"]].groupby("Dialogue_ID").count().reset_index().sort_values("Label")

In [ ]:
dimDialogAct, Contexts, Labels = Format("dyda_da", 5, "spread").get_contexts_labels()

In [ ]:
len(Contexts[0])

In [ ]:
Labels[0].shape

In [ ]:
# BERTencoder('bert-base-cased').batch_embedding(Contexts[0][:7])

## Testing Pipeline working

In [ ]:
perf = Pipeline("dyda_da", "stacked", 5, 'bert-base-cased', 3, "MLP").summary_exec()

In [ ]:
perf